## Kernel: Python 3.8

In [1]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [2]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        }
        account = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=account, headers=headers)
        token = response.json()['data'].get('access_token')
    
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    
    return token, account

(token, account) = login()
headers = dict()
headers["Accept"] = 'application/json'
headers["Authorization"] = f'Bearer {token}'
headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

print(f'login success')

login success


## 1. Fetch ALL Mitra (Raw & Test)

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
reports = response.json().get('data')

print(reports)

## 2. Process Data FULL

In [4]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

#===== List Initialization
id_position_cols = []
position_name_cols = []
mitra_name_cols = []
activity_name_cols = []
position_quota_cols = []
location_cols = []
activity_type_cols = []
logo_mitra_cols = []

# TODO: EXTRA FIELDS
start_duration_cols = []
end_duration_cols = []
start_registration_cols = []
end_registration_cols = []
credits_count_cols = []
location_code_cols = []
mitra_id_cols = []
certified_cols = []

Nomor_Mitra_cols = []
Nomor_Mitra = 1

for mitra_data_detail in mitra_data:
    #===== fetch data from mitra_data_detail dict in mitra_data list
    id_position = mitra_data_detail["id"]
    position_name = mitra_data_detail["name"]
    mitra_name = mitra_data_detail["mitra_name"]
    activity_name = mitra_data_detail["activity_name"]
    position_quota = mitra_data_detail["total"]
    location = mitra_data_detail["location"]
    activity_type = mitra_data_detail["activity_type"]
    logo_mitra = mitra_data_detail["logo"]

    # DONE: EXTRA FIELDS
    start_duration = mitra_data_detail["start_duration"]
    end_duration = mitra_data_detail["end_duration"]
    start_registration = mitra_data_detail["start_registration"]
    end_registration = mitra_data_detail["end_registration"]
    credits_count = mitra_data_detail["credits_count"]
    location_code = mitra_data_detail["location_kotakab_code"]
    mitra_id = mitra_data_detail["mitra_id"]
    certified = mitra_data_detail["certified"]

    

    #===== Append Data
    Nomor_Mitra_cols.append(Nomor_Mitra)
    id_position_cols.append(id_position)
    position_name_cols.append(position_name)
    mitra_name_cols.append(mitra_name)
    activity_name_cols.append(activity_name)
    position_quota_cols.append(position_quota)
    location_cols.append(location)
    activity_type_cols.append(activity_type)
    logo_mitra_cols.append(logo_mitra)

    # DONE: EXTRA FIELDS
    start_duration_cols.append(start_duration)
    end_duration_cols.append(end_duration)
    start_registration_cols.append(start_registration)
    end_registration_cols.append(end_registration)
    credits_count_cols.append(credits_count)
    location_code_cols.append(location_code)
    mitra_id_cols.append(mitra_id)
    certified_cols.append(certified)

    Nomor_Mitra += 1

#===== Convert Data to DataFrame
mitra_data_df_full = pd.DataFrame({
    "Nomor_Mitra": Nomor_Mitra_cols,
    "ID_Posisi": id_position_cols,
    "Nama_Posisi": position_name_cols,
    "Nama_Mitra": mitra_name_cols,
    "Nama_Kegiatan": activity_name_cols,
    "Kuota_Posisi": position_quota_cols,
    "Lokasi_Bekerja": location_cols,
    "Tipe_Aktivitas": activity_type_cols,
    "URL_Logo": logo_mitra_cols,

    # DONE: EXTRA FIELDS
    "Mulai_Periode": start_duration_cols,
    "Berakhir_Periode": end_duration_cols,
    "Mulai_Pendaftaran": start_registration_cols,
    "Berakhir_Pendaftaran": end_registration_cols,
    "SKS_Diberikan": credits_count_cols,
    "Kode_Lokasi": location_code_cols,
    "ID_Mitra": mitra_id_cols,
    "Kegiatan_Bersertifikat": certified_cols
})


## 2. Process Data COMPACT

In [4]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

Nomor_Mitra_cols = []
Nomor_Mitra = 1
id_position_cols = []
position_name_cols = []
mitra_name_cols = []
id_mitra_cols = []
activity_name_cols = []
position_quota_cols = []
location_cols = []
activity_type_cols = []
logo_mitra_cols = []

for mitra_data_detail in mitra_data:
    # fetch data from mitra_data_detail dict in mitra_data list
    id_position = mitra_data_detail["id"]
    position_name = mitra_data_detail["name"]
    mitra_name = mitra_data_detail["mitra_name"]
    id_mitra = mitra_data_detail["mitra_id"]
    activity_name = mitra_data_detail["activity_name"]
    position_quota = mitra_data_detail["total"]
    location = mitra_data_detail["location"]
    activity_type = mitra_data_detail["activity_type"]
    logo_mitra = mitra_data_detail["logo"]

    # Append Data
    Nomor_Mitra_cols.append(Nomor_Mitra)
    id_position_cols.append(id_position)
    position_name_cols.append(position_name)
    mitra_name_cols.append(mitra_name)
    id_mitra_cols.append(id_mitra)
    activity_name_cols.append(activity_name)
    position_quota_cols.append(position_quota)
    location_cols.append(location)
    activity_type_cols.append(activity_type)
    logo_mitra_cols.append(logo_mitra)

    Nomor_Mitra += 1


mitra_data_df_compact = pd.DataFrame({
    "no_mitra": Nomor_Mitra_cols,
    "id_posisi": id_position_cols,
    "nama_posisi": position_name_cols,
    "nama_mitra": mitra_name_cols,
    "id_mitra": id_mitra_cols,
    "nama_kegiatan": activity_name_cols,
    "kuota_posisi": position_quota_cols,
    "lokasi_kerja": location_cols,
    "tipe_aktivitas": activity_type_cols,
    "url_logo": logo_mitra_cols,
})


In [5]:
mitra_data_df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2409 entries, 0 to 2408
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Nomor_Mitra             2409 non-null   int64 
 1   ID_Posisi               2409 non-null   object
 2   Nama_Posisi             2409 non-null   object
 3   Nama_Mitra              2409 non-null   object
 4   Nama_Kegiatan           2409 non-null   object
 5   Kuota_Posisi            2409 non-null   int64 
 6   Lokasi_Bekerja          2409 non-null   object
 7   Tipe_Aktivitas          2409 non-null   object
 8   URL_Logo                2409 non-null   object
 9   Mulai_Periode           2409 non-null   object
 10  Berakhir_Periode        2409 non-null   object
 11  Mulai_Pendaftaran       2409 non-null   object
 12  Berakhir_Pendaftaran    2409 non-null   object
 13  SKS_Diberikan           2409 non-null   int64 
 14  Kode_Lokasi             2409 non-null   object
 15  ID_M

In [5]:
mitra_data_df_compact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   no_mitra        1954 non-null   int64 
 1   id_posisi       1954 non-null   object
 2   nama_posisi     1954 non-null   object
 3   nama_mitra      1954 non-null   object
 4   id_mitra        1954 non-null   object
 5   nama_kegiatan   1954 non-null   object
 6   kuota_posisi    1954 non-null   int64 
 7   lokasi_kerja    1954 non-null   object
 8   tipe_aktivitas  1954 non-null   object
 9   url_logo        1954 non-null   object
dtypes: int64(2), object(8)
memory usage: 152.8+ KB


In [7]:
mitra_data_df_full

,Nomor_Mitra,ID_Posisi,Nama_Posisi,Nama_Mitra,Nama_Kegiatan,Kuota_Posisi,Lokasi_Bekerja,Tipe_Aktivitas,URL_Logo,Mulai_Periode,Berakhir_Periode,Mulai_Pendaftaran,Berakhir_Pendaftaran,SKS_Diberikan,Kode_Lokasi,ID_Mitra,Kegiatan_Bersertifikat
0,1,b2806201-9170-11ed-9868-2aac6d703fc9,-,Garment Manufacturing,PAN 8 Sragen Internship Program (PSIP),1,Kab. Boyolali,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-10T07:00:00+07:00,2023-07-10T07:00:00+07:00,2022-11-21T07:00:00+07:00,2023-01-20T07:00:00+07:00,20,030900,7e6a4c10-aca9-4b78-89d5-7a56baf5c15e,True
1,2,db594c01-9170-11ed-89f0-56581bb5cdd9,-,Garment Manufacturing,PAN 8 Sragen Internship Program (PSIP),1,Kab. Boyolali,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-10T07:00:00+07:00,2023-07-10T07:00:00+07:00,2022-11-21T07:00:00+07:00,2023-01-20T07:00:00+07:00,20,030900,7e6a4c10-aca9-4b78-89d5-7a56baf5c15e,True
2,3,646376e9-962a-11ed-abf1-e61deb0742d3,1,Schoters,Transformasi Start-Up Edutech Melalui Riset da...,1,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-16T07:00:00+07:00,2023-07-01T07:00:00+07:00,2022-12-01T07:00:00+07:00,2023-02-10T07:00:00+07:00,20,016300,29817ab5-be74-4c42-a223-a04eceae0eef,True
3,4,6a7bbbab-7508-11ed-b2a2-1a198d2f4c14,2D/3D Animator Intern,BNI,BNI Digital Talent Thematic Internship,13,Kota Jakarta Barat,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-16T07:00:00+07:00,2023-06-30T07:00:00+07:00,2022-12-29T07:00:00+07:00,2023-01-20T07:00:00+07:00,20,016200,1434cd57-6295-441a-898a-7294119a96d7,True
4,5,96684230-7093-11ed-8312-feb214c74b66,3D Artist,Telkom,VR Lab Development\t\t,1,Kota Yogyakarta,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-16T07:00:00+07:00,2023-06-30T07:00:00+07:00,2022-12-19T07:00:00+07:00,2023-01-27T07:00:00+07:00,20,046000,95c5cbf2-68be-4146-9fef-f891db857f3b,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404,2405,5d9bb819-7f8d-11ed-bcbb-96627b1c4bb1,Young Banker Program Batch 2,PT Bank Mandiri (Persero) Tbk,Young Banker Program Batch 2,20,Kota Jakarta Pusat,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-20T07:00:00+07:00,2023-06-30T07:00:00+07:00,2022-12-21T07:00:00+07:00,2023-01-31T07:00:00+07:00,20,016000,875a41ac-8ad4-4a1f-9d91-e6afb053ecba,True
2405,2406,947ba8dc-5502-11ed-b209-b250481d78c3,Youth Segment Intern,XL Axiata,DYNAMIC SUSTAINABILITY PROGRAMS (OFFLINE INTER...,4,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-16T07:00:00+07:00,2023-06-30T07:00:00+07:00,2022-12-20T07:00:00+07:00,2023-01-31T07:00:00+07:00,20,016300,c6eaa287-d3a3-4dc4-92b0-c3cabfe9a698,True
2406,2407,67b505b2-6b0c-11ed-8225-d61b0e2692e7,Testing Specialist - Custody & FA System,Bank CIMB Niaga,Custody & Fund Admin System,3,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-10T07:00:00+07:00,2023-07-10T07:00:00+07:00,2022-12-20T07:00:00+07:00,2023-01-20T07:00:00+07:00,20,016300,7ee6d215-9ad2-4565-81c1-555e8efb1360,True
2407,2408,f52a4546-6b0c-11ed-a22e-c63f61abb566,Treasury Digital Interactive Communication Pr...,Bank CIMB Niaga,Treasury Digital Interactive Communication,3,Kota Jakarta Selatan,OFFLINE,https://storage.googleapis.com/kampusmerdeka_k...,2023-02-10T07:00:00+07:00,2023-07-10T07:00:00+07:00,2022-12-20T07:00:00+07:00,2023-01-20T07:00:00+07:00,20,016300,7ee6d215-9ad2-4565-81c1-555e8efb1360,True


## 3. Output Files [CSV and EXCEL]

In [5]:
def output_files(df: pd.DataFrame):
    output_types = ["CSV", "Excel"]
    
    print("=== Output File Types ===")
    
    for i in range(len(output_types)):
        format_name = output_types[i]
        print(f'{i+1}. {format_name}')
    
    choose_number = int(input("Pilih format file untuk mengekspor dataframe: ")) - 1

    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")

    if choose_number == 0:
        filename = './' + dt_string + ' - data mitra mbkm.csv'
        df.to_csv(filename, index=False)
    elif choose_number == 1:
        filename = './' + dt_string + ' - data mitra mbkm.xlsx'
        df.to_excel(filename, index=False)
    else:
        raise ValueError("Not a valid input")

    print(f"Creating {output_types[choose_number]} with a file name suffix `- data mitra mbkm`")

output_files(mitra_data_df_full)

=== Output File Types ===
1. CSV
2. Excel
Creating Excel with a file name suffix `- data mitra mbkm`


## 4. Output Files [JSON]

In [8]:
import json

def output_files_json(df: pd.DataFrame):
    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")
    filename = './' + dt_string + ' - data mitra mbkm.json'
    df.to_json(filename, orient="records")

output_files_json(mitra_data_df_compact)

## Extra: 1. Time Config

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

print("now =", now)

# dd-mm-YY H.M
dt_string = now.strftime("%d-%m-%Y %H.%M")
filename ='./' + dt_string + ' - data mitra mbkm.csv'
print("date and time =", filename)